Author: Julien Michel<br>
Email:&nbsp;&nbsp; julien.michel@ed.ac.uk

# equilibration

This notebook runs a 3 step equilibration using AMBER.

TODO) Make this MD package agnostic

In [ ]:
import BioSimSpace as BSS

In [ ]:
node = BSS.Gateway.Node("A node to equilibrate a solvated molecule.")

In [ ]:
node.addAuthor(name="Julien Michel", email="julien.michel@ed.ac.uk", affiliation="University of Edimnburgh")
node.setLicense("GPLv3")

In [ ]:
node.addInput("input", BSS.Gateway.FileSet(help="Topology and coordinate files describing the solvated system"))

node.addInput("output", BSS.Gateway.String(help="The root name of the output files."))

In [ ]:
node.addOutput("nodeoutput", BSS.Gateway.FileSet(help="The parameterised and solvated molecular system in AMBER format."))

In [ ]:
node.showControls()

# Load input

In [ ]:
system = BSS.IO.readMolecules(node.getInput("input"))

# Minimisation

In [ ]:
# Initialise a short minimisation protocol.
protocol = BSS.Protocol.Minimisation(steps=1000)

In [ ]:
process = BSS.Process.Amber(system, protocol, name="minimise")

In [ ]:
process.getConfig()

In [ ]:
process.start()

In [ ]:
process.wait()

In [ ]:
minimised = process.getSystem()

# NVT equilibration (heat up to 300 K with positional restraints on non water molecules)

In [ ]:
# TODO Extend API so can restrain everything but water
# Use loop to run until target T achieved
protocol = BSS.Protocol.Equilibration(runtime=BSS.Types.Time(0.01, "nanosecond"),
                                      temperature_start=BSS.Types.Temperature(0, "kelvin"), 
                                      temperature_end=BSS.Types.Temperature(300, "kelvin"), 
                                      restrain_backbone=True)

In [ ]:
process = BSS.Process.Amber(minimised, protocol, name="equilibrate-nvt")

In [ ]:
process.getConfig()

In [ ]:
process.start()

In [ ]:
process.wait()

In [ ]:
temps = process.getTemperature(time_series=True)

In [ ]:
# Generate a plot of volume vs temperature.
plotT = BSS.Notebook.plot(process.getTime(time_series=True),
    temps, xlabel="Time (ns)", ylabel="Temperature (K)")

In [ ]:
nvt_equilibrated = process.getSystem()

# NPT equilibration (300 K, no positional restraints)

In [ ]:
# TODO 
# Use loop to run until target density achieved
# Allow setting of target pressure from constructor
protocol = BSS.Protocol.Equilibration(runtime=BSS.Types.Time(0.01, "nanosecond"),
                                      temperature=BSS.Types.Temperature(300, "kelvin"), 
                                      ensemble="NPT")

In [ ]:
process = BSS.Process.Amber(nvt_equilibrated, protocol, name="equilibrate-npt")

## Override default protocol to wrap particle coordinates in minimum image

In [ ]:
config = process.getConfig()
print (config)

In [ ]:
config[-1] = "iwrap=1,"
config.append(' /')

In [ ]:
print(config)

In [ ]:
process.setConfig(config)

In [ ]:
process.start()

In [ ]:
process.wait()

In [ ]:
#process.getDensity()

In [ ]:
vols = process.getVolume(time_series=True)

In [ ]:
# Generate a plot of volume vs temperature.
plotV = BSS.Notebook.plot(process.getTime(time_series=True),
    vols, xlabel="Time (ns)", ylabel="Volume (A3^3)")

In [ ]:
npt_equilibrated = process.getSystem()

# Terminate node

In [ ]:
node.setOutput("nodeoutput", BSS.IO.saveMolecules(node.getInput("output"), npt_equilibrated, ["rst7"]))

In [ ]:
node.validate()